In [ ]:
!wget https://lodmedia.hb.bizmrg.com/case_files/852830/train_dataset_train.csv
!wget https://lodmedia.hb.bizmrg.com/cases/852830/%D0%9C%D0%BE%D1%81%D0%BA%D0%BE%D0%B2%D1%81%D0%BA%D0%B0%D1%8F_%D0%BE%D0%B1%D0%BB%D0%B0%D1%81%D1%82%D1%8C.zip

In [ ]:
!unzip Московская_область.zip

In [ ]:
!pip install catboost

In [ ]:
import time
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import dbscan,DBSCAN
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, SGDRegressor, RidgeClassifier, LogisticRegressionCV,Ridge,QuantileRegressor,PassiveAggressiveClassifier,PoissonRegressor
from sklearn.ensemble import ExtraTreesRegressor,ExtraTreesClassifier,RandomForestClassifier,VotingClassifier,RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor,StackingRegressor,BaggingClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.svm import LinearSVC,LinearSVR,SVR,NuSVR
from sklearn.decomposition import TruncatedSVD,PCA,FactorAnalysis,IncrementalPCA,FastICA,KernelPCA,NMF
from sklearn.preprocessing import RobustScaler,QuantileTransformer,PowerTransformer,PolynomialFeatures,KBinsDiscretizer,StandardScaler,OneHotEncoder,OrdinalEncoder,FunctionTransformer,MaxAbsScaler,MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline,FeatureUnion,TransformerMixin
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,LocalOutlierFactor
from sklearn.model_selection import train_test_split,ShuffleSplit,StratifiedShuffleSplit,TimeSeriesSplit,GridSearchCV,KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.dummy import DummyRegressor,DummyClassifier
from sklearn import set_config
import catboost
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,roc_auc_score,accuracy_score,f1_score,classification_report,recall_score,make_scorer

In [ ]:
train=pd.read_csv('/content/train_dataset_train.csv')
test=pd.read_csv('/content/sample_solution.csv')
decision=pd.read_excel('/content/decision.xlsx')
user=pd.read_csv('/content/user.csv')
userd=pd.read_csv('/content/user_decision.csv')
team=pd.read_csv('/content/team_point.csv')

In [ ]:
# перенесём данные из всех таблиц в train и test
team2=pd.DataFrame(team['team_id'].unique())
team2.columns=['team_id']
for cnt1 in range(1,7):
  for cnt2 in range(9):
    team2[str(cnt1)+'_'+str(cnt2)]=0
for cnt1 in range(1,7):
  for cnt2 in range(9):
    subteam=team[team['category_id']==cnt1]
    subteam=subteam[subteam['period']==cnt2]
    for cnt3 in team2['team_id']:
      try:
        team2.loc[team2['team_id']==cnt3,str(cnt1)+'_'+str(cnt2)]=subteam['score'][subteam['team_id']==cnt3].values
      except:
        pass
for cnt1 in range(1,7):
  for cnt2 in range(9):
    user[str(cnt1)+'_'+str(cnt2)]=0

for cnt1 in user['user_id']:
  user.loc[user['user_id']==cnt1,['1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7',
       '1_8', '2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8',
       '3_0', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '4_0',
       '4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '5_0', '5_1',
       '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8', '6_0', '6_1', '6_2',
       '6_3', '6_4', '6_5', '6_6', '6_7', '6_8']]=team2.loc[
       team2['team_id']==user['team_id'][user['user_id']==cnt1].values[0],['1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7',
       '1_8', '2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8',
       '3_0', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '4_0',
       '4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '5_0', '5_1',
       '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8', '6_0', '6_1', '6_2',
       '6_3', '6_4', '6_5', '6_6', '6_7', '6_8']]
ddict=dict(zip(decision['decision_id'],decision['decision_name']))
ddict.update({311:'311'})
userd['decision_name']=list(map(lambda x: ddict[x],userd['decision_id'].tolist()))
for cnt in range(1,9):
  train[str(cnt)]=''
for cnt in range(1,9):
  for cnt1 in train['id']:
    subuserd=userd[userd['user_id']==cnt1]
    train.loc[train['id']==cnt1,str(cnt)]=' '.join(subuserd['decision_id'][subuserd['period']==cnt].astype('int').astype('str').tolist())
for cnt in range(1,9):
  train[str(cnt)]=''
for cnt in range(1,9):
  for cnt1 in train['id']:
    subuserd=userd[userd['user_id']==cnt1]
    train.loc[train['id']==cnt1,str(cnt)]=' '.join(subuserd['decision_name'][subuserd['period']==cnt].astype('str').tolist())
for cnt1 in range(1,7):
  for cnt2 in range(9):
    train[str(cnt1)+'_'+str(cnt2)]=0

for cnt1 in train['id']:
  train.loc[train['id']==cnt1,['1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7',
       '1_8', '2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8',
       '3_0', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '4_0',
       '4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '5_0', '5_1',
       '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8', '6_0', '6_1', '6_2',
       '6_3', '6_4', '6_5', '6_6', '6_7', '6_8']]=user.loc[
       user['user_id']==cnt1,['1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7',
       '1_8', '2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8',
       '3_0', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '4_0',
       '4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '5_0', '5_1',
       '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8', '6_0', '6_1', '6_2',
       '6_3', '6_4', '6_5', '6_6', '6_7', '6_8']]
train.loc[:,['1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7',
       '1_8', '2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8',
       '3_0', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '4_0',
       '4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '5_0', '5_1',
       '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8', '6_0', '6_1', '6_2',
       '6_3', '6_4', '6_5', '6_6', '6_7', '6_8']]=train.loc[:,['1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7',
       '1_8', '2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8',
       '3_0', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '4_0',
       '4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '5_0', '5_1',
       '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8', '6_0', '6_1', '6_2',
       '6_3', '6_4', '6_5', '6_6', '6_7', '6_8']].fillna(0).astype('int')
for cnt in range(1,9):
  test[str(cnt)]=''
for cnt in range(1,9):
  for cnt1 in test['id']:
    subuserd=userd[userd['user_id']==cnt1]
    test.loc[test['id']==cnt1,str(cnt)]=' '.join(subuserd['decision_id'][subuserd['period']==cnt].astype('int').astype('str').tolist())
for cnt in range(1,9):
  test[str(cnt)]=''
for cnt in range(1,9):
  for cnt1 in test['id']:
    subuserd=userd[userd['user_id']==cnt1]
    test.loc[test['id']==cnt1,str(cnt)]=' '.join(subuserd['decision_name'][subuserd['period']==cnt].astype('str').tolist())
test['team_id']=''
test['game_id']=''
for cnt1 in train['id']:
  subuser=user[user['user_id']==cnt1]
  test.loc[test['id']==cnt1,'team_id']=' '.join(subuser['team_id'].astype('int').astype('str').tolist())
  test.loc[test['id']==cnt1,'game_id']=' '.join(subuser['game_id'].astype('int').astype('str').tolist())
for cnt1 in range(1,7):
  for cnt2 in range(9):
    test[str(cnt1)+'_'+str(cnt2)]=0

for cnt1 in test['id']:
  test.loc[test['id']==cnt1,['1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7',
       '1_8', '2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8',
       '3_0', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '4_0',
       '4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '5_0', '5_1',
       '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8', '6_0', '6_1', '6_2',
       '6_3', '6_4', '6_5', '6_6', '6_7', '6_8']]=user.loc[
       user['user_id']==cnt1,['1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7',
       '1_8', '2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8',
       '3_0', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '4_0',
       '4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '5_0', '5_1',
       '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8', '6_0', '6_1', '6_2',
       '6_3', '6_4', '6_5', '6_6', '6_7', '6_8']]
test.loc[:,['1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7',
       '1_8', '2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8',
       '3_0', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '4_0',
       '4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '5_0', '5_1',
       '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8', '6_0', '6_1', '6_2',
       '6_3', '6_4', '6_5', '6_6', '6_7', '6_8']]=test.loc[:,['1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7',
       '1_8', '2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8',
       '3_0', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '4_0',
       '4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '5_0', '5_1',
       '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8', '6_0', '6_1', '6_2',
       '6_3', '6_4', '6_5', '6_6', '6_7', '6_8']].fillna(0).astype('int')    

In [ ]:
# обучение модели
for cnt in ['Analytical thinking', 'Systemic thinking','Adaptability', 'Focus']:
  cls=catboost.CatBoostClassifier(iterations=200, 
                                learning_rate=0.1,
                                loss_function='MultiClassOneVsAll',
                                auto_class_weights='Balanced', 
                                eval_metric='TotalF1:average=Macro',
                                tokenizers=[
        {
            'tokenizer_id': 'Sense',
            'separator_type': 'BySense',
            'lowercasing': 'True',
            'token_types':['Word', 'Number', 'SentenceBreak'],
            'sub_tokens_policy':'SeveralTokens'
        }      
    ],
    dictionaries = [
        {
            'dictionary_id': 'Word',
            'max_dictionary_size': '50000'
        }
    ],
    feature_calcers = [
        'BoW:top_tokens_count=10000'
    ])
  cls.fit(train[[ '1', '2', '3', '4', '5', '6', '7', '8', 'team_id', 'game_id',
       '1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8', '2_0',
       '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8', '3_0', '3_1',
       '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '4_0', '4_1', '4_2',
       '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '5_0', '5_1', '5_2', '5_3',
       '5_4', '5_5', '5_6', '5_7', '5_8', '6_0', '6_1', '6_2', '6_3', '6_4',
       '6_5', '6_6', '6_7', '6_8']],train[cnt],
        cat_features=['team_id', 'game_id',
       '1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8', '2_0',
       '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8', '3_0', '3_1',
       '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '4_0', '4_1', '4_2',
       '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '5_0', '5_1', '5_2', '5_3',
       '5_4', '5_5', '5_6', '5_7', '5_8', '6_0', '6_1', '6_2', '6_3', '6_4',
       '6_5', '6_6', '6_7', '6_8'], 
        text_features=['1', '2', '3', '4'],
        verbose=40)
  test[cnt]=cls.predict(test[[ '1', '2', '3', '4', '5', '6', '7', '8', 'team_id', 'game_id',
       '1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8', '2_0',
       '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8', '3_0', '3_1',
       '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '4_0', '4_1', '4_2',
       '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '5_0', '5_1', '5_2', '5_3',
       '5_4', '5_5', '5_6', '5_7', '5_8', '6_0', '6_1', '6_2', '6_3', '6_4',
       '6_5', '6_6', '6_7', '6_8']])

In [ ]:
test.loc[:,['Analytical thinking', 'Systemic thinking', 'Adaptability','Focus'
]]=test.loc[:,['Analytical thinking', 'Systemic thinking', 'Adaptability','Focus']].astype('int')
submit=test[['id','Analytical thinking', 'Systemic thinking', 'Adaptability','Focus']]
submit.to_csv('submit.csv',index=False)